In [ ]:
# In[3]:
import requests
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
import time, re
from collections import defaultdict, OrderedDict
import sys
import json


def store(data):
    with open('2018-12-2 0915.json','w',encoding='utf8') as f:
        f.write(data.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding))
    

driver = webdriver.Chrome()
driver.implicitly_wait(3)
driver.get("https://news.tvbs.com.tw/politics/")
count = 0
nlist = []
for i in range(1,200):#拉
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
sp = BeautifulSoup(driver.page_source)
for j in range(0,2000):#我不知道底多少,設一個較大的數字
    try:
        first = sp.find_all("div", {"class":"real_time_box"})[0]
        two = first.find("ul")
        three = two.find_all('li')[j]#變量 0~拉到底
        four = three.find_all("div", {"class":"content_center_contxt_real_news"})[0]
    except:
        continue
    for i in range(0,6):
        try:
            five = four.select('a')[i]['href']#變量 一個li中有6個a href
            news_url = "https://news.tvbs.com.tw"+str(five)
           
            title = four.select("h2")[i].text
      
            html1 = requests.get(news_url)
            html1.encoding="utf-8"
            
            sp1 = BeautifulSoup(html1.text, 'html.parser')
            
            ct = sp1.find("div", {"class":"h7 margin_b20"}).text
            ct2 = ct.replace("\t", "")
            ct3 = ct2.replace("\n", "")
            
            da = sp1.find("div", {"class":"icon_time time leftBox2"}).text
            
            au = sp1.find("h4", {"class":"font_color5 leftBox1"}).text
            
            kw = sp1.find("div", {"class":"adWords"}).text
            
            kw1 = kw.replace("\t", "")
            kw2 = kw1.replace(",","")
            kw3 = kw2.replace("\n", "")
            
            print(title, news_url, da, au, kw3, ct3)
            count += 1
            print('第',count)
            
            newsd = {"source":"TVBS","title":title,"url":news_url,"date_":da,
                 "type":"net","author":au,"kw":kw3,"views":None,"content":ct3}
        
            nlist.append(newsd)
            
            
        except:
            continue
            
json_data = json.dumps(nlist, ensure_ascii=False, indent=4, sort_keys=True)
store(json_data)

print("下載完了!")
